# ==============================================
# 2_data_preprocessing.ipynb
# Bereinigung, Tokenisierung & Splitting für MailAssist
# ==============================================

# Zelle 1: Importiere benötigte Bibliotheken
import os
import pandas as pd
import re
import json
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize

# Stelle sicher, dass NLTK-Daten vorhanden sind (einmalig ausführen)
nltk.download('punkt')

# Definiere Pfade zu den Rohdaten und zum Speicherort der verarbeiteten Daten
RAW_DATA_DIR = "../data/raw"
PROCESSED_DATA_DIR = "../data/processed"
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Zelle 2: Rohdaten laden
# Beispiel: Die CSV-Datei "emails_raw.csv" enthält die Spalten "subject" und "body"
raw_file_path = os.path.join(RAW_DATA_DIR, "emails_raw.csv")
df = pd.read_csv(raw_file_path)
print(f"Anzahl E-Mails: {len(df)}")
display(df.head())

# Zelle 3: Datenbereinigung
def clean_text(text):
    """
    Entfernt HTML-Tags, überflüssige Leerzeichen und unerwünschte Zeichen.
    Konvertiert den Text in Kleinbuchstaben.
    """
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r"<.*?>", "", text)  # Entferne HTML-Tags
    text = re.sub(r"\s+", " ", text)     # Mehrfache Leerzeichen reduzieren
    return text.strip().lower()

# Wende die Bereinigung auf Betreff und Body an
df["clean_subject"] = df["subject"].apply(clean_text)
df["clean_body"] = df["body"].apply(clean_text)

# Optional: Entferne E-Mails, deren Body zu kurz sind (z.B. < 20 Zeichen)
df = df[df["clean_body"].str.len() > 20]

print("\nBeispiel bereinigter E-Mails:")
display(df[["clean_subject", "clean_body"]].head())

# Zelle 4: Tokenisierung
def tokenize_text(text):
    """
    Zerlegt den Text in Tokens mittels NLTK.
    """
    return word_tokenize(text)

# Tokenisiere Betreff und Body
df["subject_tokens"] = df["clean_subject"].apply(tokenize_text)
df["body_tokens"] = df["clean_body"].apply(tokenize_text)

print("\nBeispiel tokenisierter E-Mails:")
display(df[["subject_tokens", "body_tokens"]].head())

# Zelle 5: Vokabular erstellen
from collections import Counter

# Kombiniere alle Tokens aus Betreff und Body
all_tokens = []
for tokens in df["subject_tokens"]:
    all_tokens.extend(tokens)
for tokens in df["body_tokens"]:
    all_tokens.extend(tokens)

token_freqs = Counter(all_tokens)
print(f"Anzahl eindeutiger Tokens: {len(token_freqs)}")

# Beschränke das Vokabular auf die häufigsten Tokens (optional)
VOCAB_SIZE = 10000  # Du kannst diesen Wert anpassen
most_common_tokens = token_freqs.most_common(VOCAB_SIZE)

# Definiere Sondertokens
special_tokens = ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
word2id = {}
idx = 0
for token in special_tokens:
    word2id[token] = idx
    idx += 1

for token, _ in most_common_tokens:
    if token not in word2id:
        word2id[token] = idx
        idx += 1

final_vocab_size = len(word2id)
print(f"Endgültige Vokabulargröße (inkl. Sondertokens): {final_vocab_size}")

# Erstelle das inverse Mapping
id2word = {v: k for k, v in word2id.items()}

# Zelle 6: Tokens in IDs umwandeln
def tokens_to_ids(token_list, word2id, unk_id=word2id["<UNK>"]):
    """
    Wandelt eine Liste von Tokens in ihre entsprechenden IDs um.
    """
    return [word2id[t] if t in word2id else unk_id for t in token_list]

df["subject_ids"] = df["subject_tokens"].apply(lambda tokens: tokens_to_ids(tokens, word2id))
df["body_ids"] = df["body_tokens"].apply(lambda tokens: tokens_to_ids(tokens, word2id))

# Füge BOS und EOS zu den E-Mail-Texten hinzu
BOS = word2id["<BOS>"]
EOS = word2id["<EOS>"]
df["subject_ids"] = df["subject_ids"].apply(lambda ids: [BOS] + ids + [EOS])
df["body_ids"] = df["body_ids"].apply(lambda ids: [BOS] + ids + [EOS])

print("\nBeispiel für Token-IDs:")
display(df[["subject_ids", "body_ids"]].head())

# Zelle 7: Aufteilung in Trainings-, Validierungs- und Test-Datensätze
# Hier nehmen wir an, dass wir sowohl für den Betreff als auch den Body den gleichen Split verwenden.
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Zelle 8: Speichern der verarbeiteten Daten und des Vokabulars
import json

# Speichere das Vokabular als JSON
vocab_path = os.path.join(PROCESSED_DATA_DIR, "vocab.json")
with open(vocab_path, "w", encoding="utf-8") as f:
    json.dump(word2id, f, ensure_ascii=False)
print("Vokabular gespeichert unter:", vocab_path)

# Speichere die DataFrames als CSV-Dateien
train_csv_path = os.path.join(PROCESSED_DATA_DIR, "train.csv")
val_csv_path = os.path.join(PROCESSED_DATA_DIR, "val.csv")
test_csv_path = os.path.join(PROCESSED_DATA_DIR, "test.csv")

train_df.to_csv(train_csv_path, index=False)
val_df.to_csv(val_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

print("\nDaten gespeichert:")
print("Train CSV:", train_csv_path)
print("Val CSV:", val_csv_path)
print("Test CSV:", test_csv_path)

# Zelle 9: Fazit & Ausblick
print("""
Die E-Mail-Daten wurden erfolgreich bereinigt, tokenisiert und in Trainings-, Validierungs- 
und Test-Datensätze aufgeteilt. Das Vokabular wurde erstellt und als JSON-Datei gespeichert.
Im nächsten Schritt (z. B. in notebooks/3_training_demo.ipynb) wird das MailAssist-Modell 
trainiert, um automatisierte E-Mail-/Textvorschläge zu generieren.
""")
